In [68]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
homepage = 'https://www.fatsecret.com'
f = requests.get(homepage+'/Default.aspx?pa=toc&s=2')
soup = BeautifulSoup(f.text)

In [3]:
food_alphabet = []
# get link of foods in alphabet A - Z
for link in soup.find_all('a'):
    if (link.get('href').find('/Default.aspx?pa=toc&f=') == 0):
        food_alphabet.append(link.get('href'))

In [4]:
print('Food link by alphabet: ', len(food_alphabet))
food_alphabet[:5]

Food link by alphabet:  27


['/Default.aspx?pa=toc&f=a&s=2',
 '/Default.aspx?pa=toc&f=b&s=2',
 '/Default.aspx?pa=toc&f=c&s=2',
 '/Default.aspx?pa=toc&f=d&s=2',
 '/Default.aspx?pa=toc&f=e&s=2']

In [5]:
all_food_link = []

# get pages link of each food alphabet
for food_al in food_alphabet:  
    web_food_alphabet = requests.get(homepage+food_al)
    soup_alphabet = BeautifulSoup(web_food_alphabet.text)
    food_page = []
    for link in soup_alphabet.find_all('a'):
        if (link.get('href').find('/Default.aspx?pa=toc&pg=') == 0):
            food_page.append(link.get('href'))
            
    # use pages link to create all pages link from min to max page no.
    if len(food_page) != 0:     
        split_link = food_page[-1].split('&f')
        maxpage = re.findall(r'\d+', split_link[0])
        left_part = split_link[0].replace(maxpage[0], '')
        right_part = '&f' + split_link[1]
        
        if int(maxpage[0]) != 0:
            food_page = [left_part + str(page) + right_part for page in range(0,int(maxpage[0]))]
            
    # start gathering food link by alphabet from food pages link
    for food_pa in food_page: 
        web_food_link = requests.get(homepage+food_pa)
        soup_food_link = BeautifulSoup(web_food_link.text)
        
        for f_link in soup_food_link.find_all('a'):
            if (f_link.get('href').find('/calories-nutrition/usda/') == 0):
                all_food_link.append(f_link.get('href'))

In [6]:
print('Total food links: ', len(all_food_link))
all_food_link[:3]

Total food links:  6898


['/calories-nutrition/usda/abalone-(mixed-species)',
 '/calories-nutrition/usda/abalone-(mixed-species-cooked-fried)',
 '/calories-nutrition/usda/acerola-(west-indian-cherry)']

In [7]:
# Create DataFrame with columns to store food information
food_nutrion_columns = ['Name', 'Calories', 'Total Fat', 'Saturated Fat', 'Trans Fat', 'Polyunsaturated Fat', 'Monounsaturated Fat',
'Cholesterol', 'Sodium', 'Total Carbohydrate', 'Dietary Fiber', 'Sugars', 'Protein',
'Vitamin D', 'Calcium', 'Iron', 'Potassium', 'Vitamin A', 'Vitamin C']

food_webscrape_df = pd.DataFrame(columns=food_nutrion_columns)

In [25]:
for target_link in all_food_link:
    food_columns_name = []
    food_information = []
    
    all_food_alphabet = requests.get(homepage+target_link).text
    soup_all_food = BeautifulSoup(all_food_alphabet)
    
    # find only food with 100g service size
    food_link_100g = []
    for foodlink in soup_all_food.find_all('a'):
        if (foodlink.get('href').find('portionamount=100.000') > 1):
            # Get link for serving size (100 g)
             food_link_100g = foodlink.get('href')
      
    if (len(food_link_100g) != 0):
        target_food = requests.get(homepage+food_link_100g).text
        time.sleep(2)
        soup_target = BeautifulSoup(target_food)
        
        # Get food name
        food_name = target_link.replace('/calories-nutrition/usda/','')
        food_columns_name.append('Name')
        food_information.append(food_name)

        # Get calories
        calories_name = soup_target.find("div",{"class":"hero_label black"}).text
        calories = soup_target.find("div",{"class":"hero_value black right"}).text
        food_columns_name.append(calories_name)
        food_information.append(calories)

        # Get other nutritions 
        for servingsize in soup_target.findAll("div",{"class":"nutrition_facts us"}):

            main_nutri = servingsize.findAll("div",{"class":"nutrient black left"})
            sub_nutri = servingsize.findAll("div",{"class":"nutrient sub left"})  

            # Total Fat
            food_columns_name.append(main_nutri[0].text)

            # get sub nutrient of fat depend on their information contains
            if (len(sub_nutri) <= 4 ): 
                for sub in sub_nutri[0:2]:
                    food_columns_name.append(sub.text)
            else:
                for sub in sub_nutri[0:4]:
                    food_columns_name.append(sub.text)

            # Cholesterol, Sodium, Total Carbohydrate
            for main in main_nutri[1:4]:  
                food_columns_name.append(main.text)    

            # get sub nutrient of carbohydrate
            if (len(sub_nutri) <= 4 ): 
                for sub in sub_nutri[2:]:
                    food_columns_name.append(sub.text)
            else:
                for sub in sub_nutri[4:]:
                    food_columns_name.append(sub.text)   

            # Protein
            food_columns_name.append(main_nutri[-1].text)
            # Vitamin    
            for nutriname in servingsize.findAll("div",{"class":"nutrient left"}):
                if nutriname.text != '\xa0': # space
                    food_columns_name.append(nutriname.text)

            # Get all nutrient values
            for nutris in servingsize.findAll("div",{"class":"nutrient value left"}):
                food_information.append(nutris.text)

        # Insert new value to dataframe by columns names
        dataframe_index = len(food_webscrape_df)
        food_webscrape_df.loc[dataframe_index,food_columns_name] = food_information
    

In [31]:
print('Total food data: ', len(food_webscrape_df))
food_webscrape_df.head(10)

Total food data:  6897


,Name,Calories,Total Fat,Saturated Fat,Trans Fat,Polyunsaturated Fat,Monounsaturated Fat,Cholesterol,Sodium,Total Carbohydrate,Dietary Fiber,Sugars,Protein,Vitamin D,Calcium,Iron,Potassium,Vitamin A,Vitamin C
0,abalone-(mixed-species),105,0.76g,0.149g,-,0.104g,0.107g,85mg,301mg,6.01g,0g,0g,17.1g,-,31mg,3.19mg,250mg,2mcg,2mg
1,abalone-(mixed-species),105,0.76g,0.149g,-,0.104g,0.107g,85mg,301mg,6.01g,0g,0g,17.1g,-,31mg,3.19mg,250mg,2mcg,2mg
2,abalone-(mixed-species-cooked-fried),189,6.78g,1.646g,-,1.676g,2.741g,94mg,591mg,11.05g,0g,-,19.63g,-,37mg,3.8mg,284mg,2mcg,1.8mg
3,acerola-(west-indian-cherry),32,0.3g,0.068g,-,0.09g,0.082g,0mg,7mg,7.69g,1.1g,-,0.4g,-,12mg,0.2mg,146mg,38mcg,1677.6mg
4,acerola-juice,23,0.31g,0.069g,-,0.092g,0.084g,0mg,3mg,4.9g,0.3g,4.6g,0.41g,-,10mg,0.51mg,99mg,26mcg,1633.9mg
5,abiyuch,69,0.1g,0.014g,-,NaN,NaN,-,20mg,17.6g,5.3g,8.55g,1.5g,-,8mg,1.61mg,304mg,5mcg,54.1mg
6,acorn-flour-(full-fat),501,30.17g,3.923g,-,5.813g,19.11g,0mg,0mg,54.65g,-,-,7.49g,-,43mg,1.21mg,712mg,3mcg,0mg
7,adobo-fresco,226,20.9g,2.91g,-,2.044g,14.963g,0mg,17152mg,18.6g,1.3g,2.03g,2g,-,123mg,3.2mg,187mg,11mcg,5.8mg
8,acorn-winter-squash,40,0.1g,0.021g,-,0.042g,0.007g,0mg,3mg,10.42g,1.5g,-,0.8g,-,33mg,0.7mg,347mg,18mcg,11mg
9,acorn-winter-squash-(without-salt-cooked-baked),56,0.14g,0.029g,-,0.059g,0.01g,0mg,4mg,14.58g,4.4g,-,1.12g,-,44mg,0.93mg,437mg,21mcg,10.8mg


In [51]:
clean_webscrape = food_webscrape_df.copy()

In [64]:
# The nutrient that contain value '-' meaning that there are no data which can replace with 0
# Same as the NaN value that nutrient web does not provid, so it could replace with 0 
clean_webscrape = clean_webscrape.replace('-',0)
clean_webscrape = clean_webscrape.fillna(0)

In [67]:
clean_webscrape.head(5)

,Name,Calories,Total Fat,Saturated Fat,Trans Fat,Polyunsaturated Fat,Monounsaturated Fat,Cholesterol,Sodium,Total Carbohydrate,Dietary Fiber,Sugars,Protein,Vitamin D,Calcium,Iron,Potassium,Vitamin A,Vitamin C
0,abalone-(mixed-species),105,0.76g,0.149g,0,0.104g,0.107g,85mg,301mg,6.01g,0g,0g,17.1g,0,31mg,3.19mg,250mg,2mcg,2mg
1,abalone-(mixed-species),105,0.76g,0.149g,0,0.104g,0.107g,85mg,301mg,6.01g,0g,0g,17.1g,0,31mg,3.19mg,250mg,2mcg,2mg
2,abalone-(mixed-species-cooked-fried),189,6.78g,1.646g,0,1.676g,2.741g,94mg,591mg,11.05g,0g,0,19.63g,0,37mg,3.8mg,284mg,2mcg,1.8mg
3,acerola-(west-indian-cherry),32,0.3g,0.068g,0,0.09g,0.082g,0mg,7mg,7.69g,1.1g,0,0.4g,0,12mg,0.2mg,146mg,38mcg,1677.6mg
4,acerola-juice,23,0.31g,0.069g,0,0.092g,0.084g,0mg,3mg,4.9g,0.3g,4.6g,0.41g,0,10mg,0.51mg,99mg,26mcg,1633.9mg


In [69]:
clean_webscrape.to_csv('Food nutrients scrape.csv', encoding='utf-8', index=False)